# 시맨틱 검색 - Semantic Search


## 시맨틱 검색 (Semantic Search)

오픈서치의 시맨틱 검색은 검색 쿼리의 의미를 이해하고, 그에 따라 가장 관련성 높은 결과를 사용자에게 제공하는 기능입니다. 이는 전통적인 키워드 기반 검색과 달리, 검색 쿼리의 맥락과 의미를 분석하여 보다 정확하고 관련성 높은 검색 결과를 제공합니다. 예를 들어 "아마존"이라는 단어가 상품명인지, 회사명인지, 아니면 지역명인지 등 쿼리의 의미를 파악하여 그에 맞는 검색 결과를 제공합니다.


## 신경망 검색(Neural Search)

수년 동안 고객들은 OpenSearch k-NN을 기반으로 시맨틱 검색 애플리케이션을 구축하기 위해서는 텍스트 임베딩 모델을 검색 및 수집 파이프라인에 통합하기 위해 추가적인 미들웨어를 구축해야 하는 부담이 있었습니다. 이제 Amazon SageMaker와 Amazon Bedrock와의 통합을 통해 신경망 검색을 강화하며 클러스터에서 실행되는 시맨틱 검색 파이프라인을 지원할 수 있습니다.

신경망 검색은 텍스트를 벡터로 변환하고 인덱싱 시간과 검색 시간 모두에서 벡터 검색을 용이하게 합니다. 인덱싱 중에 신경망 검색은 문서 텍스트를 벡터 임베딩으로 변환하고 텍스트와 그 벡터 임베딩을 모두 벡터 인덱스에 인덱싱합니다. 신경망 쿼리를 사용하는 경우 신경망 검색은 쿼리 텍스트를 벡터 임베딩으로 변환하고, 벡터 검색을 사용하여 쿼리와 문서 임베딩을 비교한 다음 가장 가까운 결과를 반환합니다.

문서를 인덱스에 인제스트하기 전에 문서는 기계 학습(ML) 모델을 통과하게 되며, 이 모델은 문서 필드에 대한 벡터 임베딩을 생성합니다. 검색 요청을 보내면 쿼리 텍스트나 이미지도 ML 모델을 통과하여 해당 벡터 임베딩을 생성합니다. 그런 다음 신경망 검색이 임베딩에 대한 벡터 검색을 수행하고 일치하는 문서를 반환합니다.

신경망 검색을 사용하면 OpenSearch API를 통해 인간의 언어로 검색 쿼리를 실행하고, Amazon SageMaker에서 호스팅되거나 Amazon Bedrock에서 관리하는 텍스트 임베딩을 통해 의미론적 이해와 유사성을 고려한 텍스트 임베딩을 사용하여 더 정확한 결과를 제공할 수 있습니다.


## 사전준비


필요한 패키지를 설치합니다.


In [1]:
%store -r model_id

In [ ]:
!pip install -q boto3
!pip install -q requests
!pip install -q requests-aws4auth
!pip install -q opensearch-py
!pip install -q tqdm
!pip install -q boto3
!pip install -q -U pandas
!pip install -q -U tiktoken

# 데이터 준비

키워드 검색 단계에서와 마찬가지로 데이터를 준비합니다.


In [2]:
import pandas as pd
import requests

df = pd.read_csv("./data/movies.csv", low_memory=False)
df.head(5)

,title,genre,year,date,rating,vote_count,plot,main_act,supp_act
0,변호인,드라마,2013,12.18,8.99,94574,"198 년대 초 부산. 빽 없고, 돈 없고, 가방끈도 짧은 세무 변호사 송우석(송강...",송강호|김영애|오달수|곽도원|임시완,송영창|정원중|조민기|이항나|이성민|차은재|차광수|한기중|심희섭|조완기
1,어벤져스: 엔드게임,액션|SF,2019,4.24,9.38,68923,인피니티 워 이후 절반만 살아남은 지구 마지막 희망이 된 어벤져스 먼저 떠난 그들을...,로버트 다우니 주니어|크리스 에반스|크리스 헴스워스|마크 러팔로|스칼렛 요한슨|제레...,베네딕트 컴버배치|조 샐다나|크리스 프랫|채드윅 보스만|톰 홀랜드|안소니 마키|기네...
2,명량,액션|드라마,2014,7.30,8.44,66953,"1597년 임진왜란 6년, 오랜 전쟁으로 인해 혼란이 극에 달한 조선. 무서운 속도...",최민식|류승룡|조진웅,진구|이정현|김명곤|권율|노민우|김태훈|오타니 료헤이|이승준|김강일|박보검|이해영|...
3,부산행,액션|스릴러,2016,7.20,8.00,59184,"정체불명의 바이러스가 전국으로 확산되고 대한민국 긴급재난경보령이 선포된 가운데, 열...",공유|정유미|마동석|김수안|김의성|최우식|안소희,최귀화|정석용|예수정|박명신|장혁진
4,신과함께-죄와 벌,판타지|드라마,2017,12.20,7.83,58124,"저승 법에 의하면, 모든 인간은 사후 49일 동안 7번의 재판을 거쳐야만 한다. 살...",하정우|차태현|주지훈|김향기|김동욱|마동석,오달수|임원희|디오|이준혁|예수정|장광|정해균|김수안|남일우|정지훈


데이터의 스키마를 확인합니다.


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   title       1000 non-null   object 
 1   genre       1000 non-null   object 
 2   year        1000 non-null   int64  
 3   date        1000 non-null   float64
 4   rating      1000 non-null   float64
 5   vote_count  1000 non-null   int64  
 6   plot        1000 non-null   object 
 7   main_act    1000 non-null   object 
 8   supp_act    1000 non-null   object 
dtypes: float64(2), int64(2), object(5)
memory usage: 70.4+ KB


In [4]:
from tiktoken._educational import *

# Train a BPE tokeniser on a small amount of text
enc = tiktoken.get_encoding("cl100k_base")
len(enc.encode("hello world aaaaaaaaaaaa"))

5

벡터 필드에 영화에 대한 전체적인 정보를 담기 위해 전체 컬럼을 모두 조합한 `text` 컬럼을 추가합니다. 이 `text` 필드는 이후 ingest pipeline에 의해 임베딩될 필드입니다.
이 워크샵에서 사용할 임베딩 모델인 amazon.titan-embed-text-v2:0 의 Max Sequence 길이는 8096입니다. 따라서 여기에 맞게 임베딩할 plot 컬럼의 최대 길이를 8000으로 truncate합니다.

In [5]:
import pandas as pd


def create_text(row):
    text = ""
    for col, val in row.items():
        text += f"{col}: {val} | "

    encoding = tiktoken.get_encoding("cl100k_base")

    tokens = encoding.encode(text)

    # Get the number of tokens
    num_tokens = len(tokens)
    print(f"Number of tokens: {num_tokens}")

    # Truncate the text to 500 tokens
    max_tokens = 8000
    if num_tokens > max_tokens:
        truncated_tokens = tokens[:max_tokens]
        truncated_text = encoding.decode(truncated_tokens)
        print(f"Truncated text ({max_tokens} tokens): {truncated_text}")
        return truncated_text
    else:
        return text


# Assuming your DataFrame is called 'df'
df["text"] = df.apply(create_text, axis=1)
df.head(10)

Number of tokens: 550
Number of tokens: 519
Number of tokens: 657
Number of tokens: 271
Number of tokens: 736
Number of tokens: 809
Number of tokens: 916
Number of tokens: 429
Number of tokens: 507
Number of tokens: 476
Number of tokens: 426
Number of tokens: 452
Number of tokens: 305
Number of tokens: 448
Number of tokens: 522
Number of tokens: 365
Number of tokens: 463
Number of tokens: 605
Number of tokens: 395
Number of tokens: 1692
Number of tokens: 653
Number of tokens: 745
Number of tokens: 1022
Number of tokens: 595
Number of tokens: 631
Number of tokens: 425
Number of tokens: 596
Number of tokens: 485
Number of tokens: 347
Number of tokens: 356
Number of tokens: 348
Number of tokens: 529
Number of tokens: 458
Number of tokens: 1067
Number of tokens: 354
Number of tokens: 648
Number of tokens: 558
Number of tokens: 503
Number of tokens: 508
Number of tokens: 676
Number of tokens: 253
Number of tokens: 502
Number of tokens: 531
Number of tokens: 563
Number of tokens: 814
Number 

,title,genre,year,date,rating,vote_count,plot,main_act,supp_act,text
0,변호인,드라마,2013,12.18,8.99,94574,"198 년대 초 부산. 빽 없고, 돈 없고, 가방끈도 짧은 세무 변호사 송우석(송강...",송강호|김영애|오달수|곽도원|임시완,송영창|정원중|조민기|이항나|이성민|차은재|차광수|한기중|심희섭|조완기,title: 변호인 | genre: 드라마 | year: 2013 | date: 1...
1,어벤져스: 엔드게임,액션|SF,2019,4.24,9.38,68923,인피니티 워 이후 절반만 살아남은 지구 마지막 희망이 된 어벤져스 먼저 떠난 그들을...,로버트 다우니 주니어|크리스 에반스|크리스 헴스워스|마크 러팔로|스칼렛 요한슨|제레...,베네딕트 컴버배치|조 샐다나|크리스 프랫|채드윅 보스만|톰 홀랜드|안소니 마키|기네...,title: 어벤져스: 엔드게임 | genre: 액션|SF | year: 2019 ...
2,명량,액션|드라마,2014,7.30,8.44,66953,"1597년 임진왜란 6년, 오랜 전쟁으로 인해 혼란이 극에 달한 조선. 무서운 속도...",최민식|류승룡|조진웅,진구|이정현|김명곤|권율|노민우|김태훈|오타니 료헤이|이승준|김강일|박보검|이해영|...,title: 명량 | genre: 액션|드라마 | year: 2014 | date:...
3,부산행,액션|스릴러,2016,7.20,8.00,59184,"정체불명의 바이러스가 전국으로 확산되고 대한민국 긴급재난경보령이 선포된 가운데, 열...",공유|정유미|마동석|김수안|김의성|최우식|안소희,최귀화|정석용|예수정|박명신|장혁진,title: 부산행 | genre: 액션|스릴러 | year: 2016 | date...
4,신과함께-죄와 벌,판타지|드라마,2017,12.20,7.83,58124,"저승 법에 의하면, 모든 인간은 사후 49일 동안 7번의 재판을 거쳐야만 한다. 살...",하정우|차태현|주지훈|김향기|김동욱|마동석,오달수|임원희|디오|이준혁|예수정|장광|정해균|김수안|남일우|정지훈,title: 신과함께-죄와 벌 | genre: 판타지|드라마 | year: 2017...
5,군함도,액션|드라마,2017,7.26,5.29,53895,1945년 일제강점기. 경성 반도호텔 악단장 ‘강옥’(황정민)과 그의 하나뿐인 딸 ...,황정민|소지섭|송중기|이정현|김수안,이경영|김민재|김중희|김인우|신승환|백승철|윤경호|장성범,title: 군함도 | genre: 액션|드라마 | year: 2017 | date...
6,26년,액션,2012,11.29,6.40,51885,"광주 수호파 중간보스 곽진배, 국가대표 사격선수 심미진, 서대문소속 경찰 권정혁, ...",진구|한혜진|임슬옹,배수빈|이경영|장광|이미도|조덕제|김의성|안석환|민복기|구성환|김민재|최귀화|김태수|장영,title: 26년 | genre: 액션 | year: 2012 | date: 11...
7,인터스텔라,SF,2014,11.06,9.10,50721,세계 각국의 정부와 경제가 완전히 붕괴된 미래가 다가온다. 지난 2 세기에 범한 잘...,매튜 맥커너히|앤 해서웨이|마이클 케인|제시카 차스테인,케이시 애플렉|웨스 벤틀리|토퍼 그레이스|매켄지 포이|엘렌 버스틴|존 리스고|빌 어...,title: 인터스텔라 | genre: SF | year: 2014 | date: ...
8,택시운전사,드라마|가족,2017,8.02,9.03,49661,택시운전사 만섭(송강호)은 외국손님을 태우고 광주에 갔다 통금 전에 돌아오면 밀린 ...,송강호|토마스 크레취만|유해진|류준열,박혁권|최귀화|차순배|신담수|류성현|엄태구|박민희|이정은|유은미|권순준|윤석호|허정...,title: 택시운전사 | genre: 드라마|가족 | year: 2017 | da...
9,극한직업,코미디,2019,1.23,8.50,47522,"불철주야 달리고 구르지만 실적은 바닥, 급기야 해체 위기를 맞는 마약반! 더 이상 ...",류승룡|이하늬|진선규|이동휘|공명,신하균|오정세|김의성|송영규|양현민|허준석|장진희|김종수|이중옥,title: 극한직업 | genre: 코미디 | year: 2019 | date: ...


### OpenSearch 도메인에 연결


In [6]:
def get_cfn_outputs(stackname, cfn):
    outputs = {}
    for output in cfn.describe_stacks(StackName=stackname)["Stacks"][0]["Outputs"]:
        outputs[output["OutputKey"]] = output["OutputValue"]
    return outputs

In [7]:
import boto3, json

# region_name = "us-west-2"
session = boto3.Session()
region_name = session.region_name

cfn = boto3.client("cloudformation", region_name)
kms = boto3.client("secretsmanager", region_name)

stackname = "opensearch-workshop"
cfn_outputs = get_cfn_outputs(stackname, cfn)

aos_credentials = json.loads(
    kms.get_secret_value(SecretId=cfn_outputs["OpenSearchSecret"])["SecretString"]
)

aos_host = cfn_outputs["OpenSearchDomainEndpoint"]
aos_host

'search-opensearch-workshop-rgx5p7rgxyxhec6mi3kusldapu.us-west-2.es.amazonaws.com'

In [8]:
from opensearchpy import OpenSearch, RequestsHttpConnection, AWSV4SignerAuth

auth = (aos_credentials["username"], aos_credentials["password"])

aos_client = OpenSearch(
    hosts=[{"host": aos_host, "port": 443}],
    http_auth=auth,
    use_ssl=True,
    verify_certs=True,
    connection_class=RequestsHttpConnection,
)

## 데이터 임베딩을 위한 Ingest Pipeline 생성


모델 배포 과정을 참고하여 배포된 모델의 ID를 확인하고 아래와 같이 변수에 초기화합니다.


In [9]:
import requests

# search_model = {"query": {"match": {"name": "OpenSearch-Cohere"}}, "size": 10}

# response = requests.get(
#     "https://" + aos_host + "/_plugins/_ml/models/_search", auth=auth, json=search_model
# )
# model_info = json.loads(response.text)
# model_id = model_info["hits"]["hits"][0]["_id"]
# model_id

## 인제스트 파이프라인 생성

영화의 줄거리 정보를 가진 특정 필드("plot")에 대한 벡터 임베딩을 생성하는 파이프라인을 설정합니다. 이러한 임베딩은 검색 인덱스 필드("vector_field")에 저장되며, 효율적인 유사성 검색 및 검색 작업에 사용될 수 있습니다.


In [10]:
pipeline = {
    "description": "Titan Text V2 Embedding",
    "processors": [
        {
            "text_embedding": {
                "model_id": model_id,
                "field_map": {
                    "text": "vector_field",
                },
            }
        }
    ],
}

pipeline_id = "movie_plot_embedding_pipeline"
aos_client.ingest.delete_pipeline(id=pipeline_id)
aos_client.ingest.put_pipeline(id=pipeline_id, body=pipeline)

{'acknowledged': True}

## 인덱스 생성

movie_semantic 인덱스를 생성합니다. 아래 세팅 및 맵핑 정보 중 중요한 것은 다음과 같습니다.

-   index.knn: KNN 검색을 위해 True로 설정합니다.
-   default_pipeline: 위 단계에서 생성한 pipeline_id를 제공합니다.
-   index.knn.space_type: 임베딩 벡터끼리의 유사도를 파악할 때 사용할 알고리즘을 cosinesimil로 지정합니다


In [11]:
index_name = "movie_semantic"

# aos_client.indices.delete(index=index_name)

movie_semantic = {
    "settings": {
        "max_result_window": 15000,
        "analysis": {"analyzer": {"analysis-nori": {"type": "nori", "stopwords": "_korean_"}}},
        "index.knn": True,
        "default_pipeline": pipeline_id,
        "index.knn.space_type": "l2",
    },
    "mappings": {
        "properties": {
            "date": {
                "type": "float",
            },
            "genre": {
                "type": "text",
            },
            "main_act": {
                "type": "text",
                "fields": {"keyword": {"type": "keyword", "ignore_above": 256}},
            },
            "plot": {
                "type": "text",
            },
            "rating": {"type": "float"},
            "supp_act": {
                "type": "text",
                "fields": {"keyword": {"type": "keyword", "ignore_above": 256}},
            },
            "title": {
                "type": "text",
                "fields": {"keyword": {"type": "keyword", "ignore_above": 256}},
            },
            "vote_count": {"type": "long"},
            "text": {
                "type": "text",
            },
            "year": {"type": "long"},
            "vector_field": {
                "type": "knn_vector",
                "dimension": 1024,
                "method": {"name": "hnsw", "space_type": "l2", "engine": "faiss"},
                "store": True,
            },
        }
    },
}

aos_client.indices.create(index=index_name, body=movie_semantic)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'movie_semantic'}

## 데이터 인제스트

키워드 검색과 동일하게 데이터를 인제스트합니다. 위 단계에서 인덱스를 생성할 때 INGEST PIPELINE을 설정했기 때문에 직접 임베딩 모델을 호출하여 데이터를 벡터로 변환하지 않아도 됩니다. 단, 데이터가 인제스트될 때 임베딩 모델을 호출해야 하는 단계가 있기 때문에 parallel_bulk의 병렬도가 높으면 에러가 발생할 있습니다. 여기서는 `thread_count`와 `queue_size`를 각각 1로 낮춰줍니다. 이 단계가 완료되는데는 약 4~5분 정도 소요됩니다.


In [12]:
from tqdm import tqdm
from opensearchpy import helpers

json_data = df.to_json(orient="records", lines=True)
docs = json_data.split("\n")[:-1]  # To remove the last empty line


def _generate_data():
    for doc in docs:
        yield {"_index": index_name, "_source": doc}


succeeded = []
failed = []
for success, item in helpers.parallel_bulk(
    aos_client, actions=_generate_data(), chunk_size=10, thread_count=1, queue_size=1
):
    if success:
        succeeded.append(item)
    else:
        failed.append(item)

데이터 인제스트가 잘 마무리되었는지 확인합니다.


In [13]:
# Refresh the index to make the changes visible
aos_client.indices.refresh(index=index_name)

count = aos_client.count(index=index_name)
print(count)

{'count': 1000, '_shards': {'total': 5, 'successful': 5, 'skipped': 0, 'failed': 0}}


# 키워드 검색과 시맨틱 검색 결과 비교


## 키워드 검색을 위한 함수 생성

키워드 단계에서 정의한 키워드 검색 함수와 동일한 함수를 생성합니다.


In [14]:
def keyword_search(query_text):
    query = {
        "size": 10,
        "_source": {"excludes": ["text", "vector_field"]},
        "query": {
            "multi_match": {
                "query": query_text,
                "fields": ["title", "plot", "genre", "main_act"],
            }
        },
    }

    res = aos_client.search(index=index_name, body=query)

    query_result = []
    for hit in res["hits"]["hits"]:
        row = [
            hit["_score"],
            hit["_source"]["title"],
            hit["_source"]["plot"],
            hit["_source"]["genre"],
            hit["_source"]["rating"],
            hit["_source"]["main_act"],
        ]
        query_result.append(row)

    query_result_df = pd.DataFrame(
        data=query_result, columns=["_score", "title", "plot", "genre", "rating", "main_act"]
    )
    display(query_result_df)

## 시맨팀 검색을 위한 함수 생성

뉴럴 검색을 위한 함수를 생성합니다. 주목해야할 부분은 다음과 같습니다.

1. **`"_source": {"excludes": ["vector_field"]}`**: 검색 결과에서 **`vector_field`** 필드를 제외한 모든 필드를 반환합니다. 이는 벡터 데이터가 크기 때문에 전송 비용을 줄이기 위함입니다.
2. **`"query": { ... }`**: 실제 검색 쿼리를 정의합니다.
3. **`"neural": { ... }`**: 벡터 검색을 수행하기 위한 쿼리 유형입니다.
4. **`"vector_field": "vector_field_name"`**: 벡터 데이터가 저장된 필드 이름입니다.
5. **`"query_text": query_text`**: 검색할 텍스트 쿼리입니다.
6. **`"model_id": model_id`**: 벡터 임베딩을 생성하는 데 사용된 모델의 ID입니다.


In [97]:
def semantic_search(query_text):
    query = {
        "size": 10,
        "_source": {"excludes": ["text", "vector_field"]},
        "query": {
            "neural": {"vector_field": {"query_text": query_text, "model_id": model_id, "k": 10}},
        },
    }

    res = aos_client.search(index=index_name, body=query)

    query_result = []
    for hit in res["hits"]["hits"]:
        row = [
            hit["_score"],
            hit["_source"]["title"],
            hit["_source"]["plot"],
            hit["_source"]["genre"],
            hit["_source"]["rating"],
            hit["_source"]["year"],
            hit["_source"]["main_act"],
        ]
        query_result.append(row)

    query_result_df = pd.DataFrame(
        data=query_result,
        columns=["_score", "title", "plot", "genre", "rating", "year", "main_act"],
    )
    display(query_result_df)

## 결과 비교

자연어 기반의 쿼리를 작성하고 키워드 검색과 시맨틱 검색의 결과를 비교해봅니다


In [98]:
query_text = "우주에서 벌어지는 전쟁 이야기를 다룬 영화 소개해줘"

In [99]:
keyword_search(query_text)

,_score,title,plot,genre,rating,main_act
0,7.071436,영화는 영화다,영화를 촬영하던 배우 장수타(강지환 扮)는 액션씬에서 욱하는 성질을 참지 못해 상대...,액션|범죄|드라마|느와르,8.85,소지섭|강지환
1,6.296067,황금나침반,‘살아 있는 모든 존재를 멸망으로 이끌 전쟁이 일어날 것’이라는 예언의 중심에 있는...,판타지|모험|액션|드라마|가족|스릴러,6.25,다니엘 크레이그|니콜 키드먼|에바 그린|다코타 블루 리차드
2,6.086336,더 넌,루마니아의 젊은 수녀가 자살하는 사건을 의뢰 받아 바티칸에서 파견된 버크 신부와 아...,공포|미스터리|스릴러,6.66,타이사 파미가|데미안 비쉬어|보니 아론스
3,5.492523,명탐정 코난: 천공의 난파선,명쾌한 추리와 시원한 액션으로 사건을 해결했던 코난. 이번에는 세계 최대 비행선에서...,애니메이션|모험|가족,9.23,타카야마 미나미|야마자키 와카나|야마구치 캇페이|코야마 리키야|하야시바라 메구미|호...
4,5.421100,그랜 토리노,자동차 공장에서 은퇴한 채 무료한 일상을 보내는 월트(클린트 이스트우드). 한국전 ...,범죄|드라마,9.23,클린트 이스트우드|크리스토퍼 칼리|비 방|어니 허
5,5.275989,오빠생각,"전쟁으로 소중한 가족도, 지켜야 할 동료도 모두 잃은 군인 ‘한상렬’(임시완). 우...",드라마|전쟁,8.51,임시완|고아성
6,5.272145,인사이드 아웃,모든 사람의 머릿속에 존재하는 감정 컨트롤 본부 그곳에서 불철주야 열심히 일하는 기...,애니메이션|코미디,9.07,에이미 포엘러|필리스 스미스|민디 캘링|빌 헤이더|루이스 블랙|케이틀린 디아스|카일...
7,5.218224,어린왕자,친구하나 없이 엄마(레이첼 맥아담스)가 짜놓은 인생계획표대로만 살던 소녀(맥켄지 포...,애니메이션|판타지,8.65,제프 브리지스|레이첼 맥아담스|매켄지 포이|마리옹 꼬띠아르|제임스 프랭코|베니시오 ...
8,4.934820,곡성(哭聲),낯선 외지인(쿠니무라 준)이 나타난 후 벌어지는 의문의 연쇄 사건들로 마을이 발칵 ...,미스터리|스릴러|드라마,7.61,곽도원|황정민|쿠니무라 준|천우희|김환희
9,4.913109,미쓰 와이프,"뉴욕 본사 발령을 앞두고, 연우(엄정화)는 갑작스러운 교통사고를 당한다. 생사의 위...",코미디,8.94,엄정화|송승헌


In [100]:
semantic_search(query_text)
# print(index_name)

,_score,title,plot,genre,rating,year,main_act
0,0.526351,우주 전쟁,레이 페리어(톰 크루즈 분)는 이혼한 항만 근로자로 아무런 희망 없이 매일을 살아간...,SF|드라마|스릴러,7.31,2005,톰 크루즈
1,0.409978,워크래프트: 전쟁의 서막,"두 개의 운명, 하나의 세계 공존할 것인가! 맞설 것인가! 서로 다른 차원에 살고 ...",액션|모험|판타지,8.66,2016,트래비스 핌멜|벤 포스터|폴라 패튼|도미닉 쿠퍼|클랜시 브라운|토비 켑벨
2,0.409093,평양성,"‘황산벌’전투를 기억하시는가? 그 후 8년, 백제를 손안에 넣은 신라가 이번엔 고구...",코미디|전쟁,7.77,2011,정진영|이문식|류승룡|윤제문|선우선
3,0.408936,혹성탈출: 종의 전쟁,전 세계에 퍼진 치명적인 바이러스 ‘시미안 플루’로 인해 유인원들은 나날이 진화하는...,액션|모험|드라마|SF,7.92,2017,앤디 서키스|우디 해럴슨|스티브 잔|아미아 밀러
4,0.403048,싸움의 기술,"맞고 사는 게 일과인, 쉼 없이 구타를 유발시키는 소심한 부실고딩 송병태. 안 맞고...",코미디|액션,7.92,2006,백윤식|재희
5,0.403024,월-E,텅 빈 지구에 홀로 남아 수백 년이란 시간을 외롭게 일만 하며 보내던 월-E (WA...,애니메이션|SF|가족|코미디|멜로/로맨스|모험,9.41,2008,벤 버트|엘리사 나이트|제프 갈린|프레드 윌러드
6,0.400586,터미네이터: 미래전쟁의 시작,"21세기 초, 군사방위 프로그램으로 제작된 네트워크 ‘스카이넷’은 자각력이 생겨 인...",SF|스릴러|액션|모험,8.18,2009,크리스찬 베일|샘 워싱턴|안톤 옐친|문 블러드굿
7,0.399407,지.아이.조 - 전쟁의 서막,타고난 재능과 강인한 결단력을 지닌 특수부대 대위 ‘듀크’는 가공할 파괴력의 최첨단...,액션|모험|SF|스릴러,7.65,2009,채닝 테이텀|데니스 퀘이드|이병헌|조셉 고든 레빗|시에나 밀러|레이첼 니콜스|크리스...
8,0.398165,1917,제1차 세계대전이 한창인 1917년. 독일군에 의해 모든 통신망이 파괴된 상황 속에...,드라마|전쟁,8.88,2020,조지 맥케이|딘-찰스 채프먼
9,0.397724,트랜스포머: 패자의 역습,"샘 윗윅키(샤이아 라보프)가 오토봇과 디셉티콘, 두 로봇 진영간의 치열한 싸움에서 ...",SF|액션|모험,8.11,2009,샤이아 라보프|메간 폭스|휴고 위빙|조쉬 더하멜|존 터투로


이전 키워드 검색에서와 동일한 방법으로 검색 결과를 최적화해봅시다. 고객 평점이 높고 최신에 나온 영화에게 높은 점수를 주어 더 검색 결과의 상위에 랭크되도록 합니다.

In [129]:
def semantic_search_enhanced(query_text):
    # query = {
    #     "size": 10,
    #     "_source": {"excludes": ["text", "vector_field"]},
    #     "query": {},
    # }

    query = {
        "size": 10,
        "query": {
            "function_score": {
                "query": {
                    "neural": {
                        "vector_field": {"query_text": query_text, "model_id": model_id, "k": 10}
                    },
                },
                "functions": [
                    {
                        "field_value_factor": {
                            "field": "rating",
                            "factor": 0.1,
                            "modifier": "log1p",
                            "missing": 5,
                        }
                    },
                    {"gauss": {"year": {"origin": 2020, "offset": 3, "scale": 1, "decay": 0.9}}},
                ],
                # "boost_mode": "multiply",
                "score_mode": "sum",
            }
        },
        "sort": ["_score", {"rating": "desc"}],
    }

    res = aos_client.search(index=index_name, body=query)

    query_result = []
    for hit in res["hits"]["hits"]:
        row = [
            hit["_score"],
            hit["_source"]["title"],
            hit["_source"]["plot"],
            hit["_source"]["genre"],
            hit["_source"]["rating"],
            hit["_source"]["year"],
            hit["_source"]["main_act"],
        ]
        query_result.append(row)

    query_result_df = pd.DataFrame(
        data=query_result,
        columns=["_score", "title", "plot", "genre", "rating", "year", "main_act"],
    )
    display(query_result_df)

In [130]:
semantic_search_enhanced(query_text)

,_score,title,plot,genre,rating,year,main_act
0,0.512536,혹성탈출: 종의 전쟁,전 세계에 퍼진 치명적인 바이러스 ‘시미안 플루’로 인해 유인원들은 나날이 진화하는...,액션|모험|드라마|SF,7.92,2017,앤디 서키스|우디 해럴슨|스티브 잔|아미아 밀러
1,0.508060,1917,제1차 세계대전이 한창인 1917년. 독일군에 의해 모든 통신망이 파괴된 상황 속에...,드라마|전쟁,8.88,2020,조지 맥케이|딘-찰스 채프먼
2,0.496278,위대한 쇼맨,"쇼 비즈니스의 창시자이자, 꿈의 무대로 전세계를 매료시킨 남자 ‘바넘’의 이야기에서...",드라마|뮤지컬,9.39,2017,휴 잭맨|잭 에프론|미셸 윌리엄스|레베카 퍼거슨|젠데이아 콜먼
3,0.495229,핵소 고지,비폭력주의자인 도스(앤드류 가필드)는 전쟁으로부터 조국과 소중한 사람들을 지키기 위...,드라마|전쟁|멜로/로맨스,9.22,2017,앤드류 가필드|샘 워싱턴|휴고 위빙|테레사 팔머|빈스 본|루크 브레이시
4,0.491870,봉오동 전투,1919년 3.1운동 이후 봉오동 일대에서 독립군의 무장항쟁이 활발해진다. 일본은 ...,액션|드라마,7.91,2019,유해진|류준열|조우진
5,0.490724,공조,비밀리에 제작된 위조 지폐 동판을 탈취하려는 내부 조직에 의해 작전 중 아내와 동료...,액션,8.27,2017,현빈|유해진|김주혁
6,0.489482,공범자들,2 8년 미국산 쇠고기 수입문제 보도로 MB정부가 큰 타격을 입자 본격적인 언론 장...,다큐멘터리,9.10,2017,이명박|김재철|김장겸|고대영
7,0.489020,미쓰백,스스로를 지키려다 어린 나이에 전과자가 되어 외롭게 살아가던 ‘백상아’ 누구도 믿지...,드라마,9.09,2018,한지민|김시아|이희준
8,0.487828,헌터 킬러,미 국방부는 격침당한 잠수함의 행방을 찾기 위해 ‘헌터 킬러’를 극비리에 투입시키고...,액션|스릴러,9.37,2018,제라드 버틀러|게리 올드만
9,0.487128,레지던트 이블: 파멸의 날,"엄브렐라가 개발한 치명적인 T-바이러스가 전 세계에 퍼지고, 바이러스에 감염된 언데...",액션|SF|스릴러,8.38,2017,밀라 요보비치|알리 라터|이아인 글렌|숀 로버츠


In [ ]:
%store model_id
%store index_name